![alt text](https://www.mbari.org/wp-content/uploads/2014/11/logo-mbari-3b.png "MBARI")
<div align="center">Copyright (c) 2020, MBARI</div>

* Distributed under the terms of the GPL License
* Maintainer: dcline@mbari.org
* Author: Danelle Cline dcline@mbari.org

## Basic Exploration of Blue whale A call

Here we will be covering a basic exploration of the blue A call.

Public sound libraries:
* [https://www.mbari.org/soundscape-listening-room/](https://www.mbari.org/soundscape-listening-room/)
* [http://cetus.ucsd.edu/voicesinthesea_org/species/baleenWhales/blue.html#](http://cetus.ucsd.edu/voicesinthesea_org/species/baleenWhales/blue.html)

## Install dependencies
First, let's install dependencies and include all packages used in this tutorial. This only needs to be done once for the duration of this notebook.

In [ ]:
%pip install boto3 --quiet
%pip install librosa==0.8.1 --quiet
%pip install matplotlib --quiet

In [ ]:
%matplotlib inline
import boto3
from botocore import UNSIGNED
from botocore.client import Config
import librosa
import numpy as np
import matplotlib.pyplot as plt
import soundfile as sf
from pathlib import Path
from librosa.display import specshow, waveplot

# Blue A Call

Blue whale A calls typical characteristics:
*  In the frequency band 70 and 120Hz
*  Less than 25 seconds long
*  Characterized by a series of short pulses

## Open an example Blue whale A call sound file

In [ ]:
# First, let's download the data used in this notebook
bucket = 'emso-tsc2021-session3-eu-west-3'
wav_filename = 'blue_A_2.wav'

s3 = boto3.resource('s3',
    aws_access_key_id='',
    aws_secret_access_key='',
    config=Config(signature_version=UNSIGNED))

# only download if needed
if not Path(wav_filename).exists():
    print('Downloading')
    s3.Bucket(bucket).download_file(wav_filename, wav_filename)
    print(f'Done downloading {wav_filename}')

samples, sample_rate = sf.read(wav_filename,dtype='float32')
nsec = (samples.size)/sample_rate # number of seconds in vector
print(f'Read {nsec} seconds of data')


##  Display waveform and STFT power

In [ ]:
num_fft = 128
overlap = 0.50
A_stft = np.abs(librosa.stft(y=samples, n_fft=num_fft,
                             hop_length=int(num_fft*(1-overlap))))

plt.figure(figsize=(10,5)) 
plt.title('Blue Whale A Call')
plt.subplot(3, 1, 1)
specshow(A_stft, x_axis='time', y_axis='hz',cmap='Blues',
                         hop_length=int(num_fft*(1-overlap)), 
                         sr=sample_rate)
plt.title('Linear-frequency Power Spectrogram');

plt.subplot(3, 1, 2)
specshow(A_stft, x_axis='time', y_axis='log', cmap='Blues',
                         hop_length=int(num_fft*(1-overlap)),
                        sr=sample_rate)
plt.title('Log-frequency Power Spectrogram');

plt.subplot(3, 1, 3)
waveplot(y=samples, sr=sample_rate)
plt.title('Raw Waveform');

##  Zoom in to the call around the frequency window

The calls occur within a narrow frequency window, so let's zoom in to take a closer look. Here, we will use different fft parameters to generate a spectrogram more suited for classification.

In [ ]:
def zoom_call(D, low_freq:int, high_freq:int, sample_rate:int, overlap:float, title:str):
    """
    Utility to zoom into the stft within a frequency bounds and plot on a linear scale
    :param D:
    :param low_freq:
    :param high_freq:
    :param sample_rate:
    :param overlap:
    :param title:
    :return:
    """
    freq_bin = float(D.shape[0]) / float(sample_rate / 2)
    minM = -1 * (D.shape[0] - int(low_freq * freq_bin))
    maxM = -1 * (D.shape[0] - int(high_freq * freq_bin))
    
    librosa.display.specshow(D[minM:maxM], x_axis='time', y_axis='hz', cmap='Blues', 
                             hop_length=int(num_fft*(1-overlap)), sr=sample_rate)
    plt.title(title);
    # Remap the y ticks
    locs, labels = plt.yticks()
    r = np.arange(start=low_freq, stop=high_freq+1, step=(high_freq - low_freq)/(len(locs) - 1))
    new_labels = ["%.2f" % i for i in r]
    plt.yticks(locs, new_labels);

In [ ]:
# Compute STFT
num_fft = 1024
overlap = 0.95
A_stft = np.abs(librosa.stft(y=samples, n_fft=num_fft,
                             hop_length=int(num_fft*(1-overlap)))) 

# Set bounds for call and plot
low_freq = 70
high_freq = 100 
title = 'Blue Whale A Call Linear-frequency Power Spectrogram'
zoom_call(A_stft, low_freq, high_freq, sample_rate, overlap,  title)

# Questions or comments
* Danelle Cline dcline@mbari.org